In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from spacy.matcher import Matcher
import networkx as nx
import tensorflow_hub as hub
import time
from tqdm import tqdm
import math
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 120)
path = os.getcwd()
# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
embed = hub.load(path+'\\model')

Vectorisation

In [ ]:
# ABSTRACT VECTORISATION
df = pickle.load(open(path+'\\cleaned_data_no_null.sav', 'rb'))
abstract_vec = np.zeros(shape=(1, 512))
for i in tqdm(range(math.ceil(len(df)/100))):
    if (i < math.ceil(len(df)/100)-1):
        my_abstract = df.loc[(i*100):((i+1)*100-1), 'abstract']
    else:
        my_abstract = df.loc[(i*100):len(df), 'abstract']
    embed_vectors = embed(my_abstract.values).numpy()
    abstract_vec = np.append(abstract_vec, embed_vectors, axis=0)

abstract_vec = np.delete(abstract_vec, 0, axis=0)

In [13]:
pickle.dump(abstract_vec, open(path+'\\abstract_vectors.sav', 'wb'))

Model

In [4]:
abstract_vec = pickle.load(open(path+'\\abstract_vectors.sav', 'rb'))

In [5]:
question = "best covid"
question = pd.Series(question)
question_vec = embed(question.values).numpy()

cosine TODO reduce memory usage 1. Approximate Algorithm or 2. 分步计算

In [6]:
len(abstract_vec)

160415

In [14]:
embed_vectors = np.append(abstract_vec, question_vec, axis=0)
similarity_matrix = cosine_similarity(embed_vectors)
similarity_matrix

MemoryError: Unable to allocate 192. GiB for an array with shape (160416, 160416) and data type float64

In [24]:
def plot_similarity(labels, features, rotation):
    corr = np.inner(features, features[0:1])
    print(corr)
    sns.set(font_scale=1.2)
    g = sns.heatmap(
        corr,
        yticklabels=labels,
        vmin=0,
        vmax=1,
        cmap="YlOrRd")

def run_and_plot(messages_):
    message_embeddings_ = embed(messages_)
    plot_similarity(messages_, message_embeddings_, 90)

In [23]:
df = pickle.load(open(path+'\\cleaned_data_no_null.sav', 'rb'))['abstract']

In [25]:
messages = ['best covid treatment']
for i in range(len(df)):
    messages.append(df[i])

In [ ]:
run_and_plot(messages)